In [5]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [6]:
from google.colab import files
uploaded = files.upload()


Saving Amprion.csv to Amprion (1).csv


In [17]:
''' Loading data '''
import pandas as pd
import io
dataframe = pd.read_csv(io.BytesIO(uploaded['Amprion.csv']))
dataframe=dataframe.loc[:, dataframe.columns != 'Date']
# path = "https://github.com/BhavithaKatari/l4g-198W1D8805/blob/main/50Hertz.csv"
# series = pd.read_csv('path',index_col="Date")
dataframe.head()

,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,02:00:00,02:15:00,02:30:00,02:45:00,03:00:00,03:15:00,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,05:00:00,05:15:00,05:30:00,05:45:00,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,07:15:00,07:30:00,07:45:00,08:00:00,08:15:00,08:30:00,08:45:00,09:00:00,09:15:00,09:30:00,09:45:00,...,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,15:15:00,15:30:00,15:45:00,16:00:00,16:15:00,16:30:00,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00,18:00:00,18:15:00,18:30:00,18:45:00,19:00:00,19:15:00,19:30:00,19:45:00,20:00:00,20:15:00,20:30:00,20:45:00,21:00:00,21:15:00,21:30:00,21:45:00,22:00:00,22:15:00,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00
0,5,5,6,7,6,5,5,5,5,5,4,4,4,4,4,5,6,6,5,7,8,8,9,9,9,8,8,7,6,6,4,3,3,3,4,5,8,9,11,12,...,13,13,14,15,16,16,19,20,23,24,25,27,27,27,27,28,32,38,42,47,51,57,61,62,66,70,75,77,76,72,72,69,66,67,66,69,68,65,61,64
1,62,60,55,55,54,52,51,50,49,45,44,45,45,43,45,46,45,47,50,50,51,51,53,51,47,43,39,31,23,17,15,13,12,13,13,15,20,22,23,24,...,25,26,27,28,27,25,28,26,27,25,24,22,22,21,21,22,26,30,31,34,34,37,40,40,40,40,38,36,35,34,34,33,31,31,31,31,31,31,27,28
2,29,29,29,30,28,27,28,26,22,20,19,18,19,19,17,16,15,14,16,17,17,16,16,15,14,15,12,8,6,4,2,1,1,1,1,1,2,3,3,3,...,3,2,3,3,4,5,5,6,7,9,11,11,13,13,14,16,22,23,23,28,29,32,34,37,38,36,37,35,34,34,30,27,28,28,25,23,21,19,16,15
3,13,12,12,11,12,12,12,10,10,10,10,10,9,11,11,11,9,9,11,11,10,11,11,11,10,9,8,6,5,4,3,3,4,4,5,5,6,8,7,7,...,13,13,13,14,14,11,11,9,8,9,9,8,9,11,9,9,9,11,14,19,21,23,22,22,22,25,27,26,24,22,21,22,22,22,23,23,24,25,24,23
4,21,20,19,16,15,14,13,11,11,11,10,10,11,11,11,11,10,11,13,14,15,17,18,18,19,20,19,16,13,10,9,6,5,4,4,4,4,4,4,5,...,5,5,8,9,9,14,15,19,21,24,29,22,21,20,18,18,18,15,16,14,13,23,30,29,28,31,36,32,31,31,34,33,31,31,31,33,33,34,33,32


In [18]:
dataframe = dataframe[0:398]
dataframe

,00:00:00,00:15:00,00:30:00,00:45:00,01:00:00,01:15:00,01:30:00,01:45:00,02:00:00,02:15:00,02:30:00,02:45:00,03:00:00,03:15:00,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,05:00:00,05:15:00,05:30:00,05:45:00,06:00:00,06:15:00,06:30:00,06:45:00,07:00:00,07:15:00,07:30:00,07:45:00,08:00:00,08:15:00,08:30:00,08:45:00,09:00:00,09:15:00,09:30:00,09:45:00,...,14:00:00,14:15:00,14:30:00,14:45:00,15:00:00,15:15:00,15:30:00,15:45:00,16:00:00,16:15:00,16:30:00,16:45:00,17:00:00,17:15:00,17:30:00,17:45:00,18:00:00,18:15:00,18:30:00,18:45:00,19:00:00,19:15:00,19:30:00,19:45:00,20:00:00,20:15:00,20:30:00,20:45:00,21:00:00,21:15:00,21:30:00,21:45:00,22:00:00,22:15:00,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00
0,5,5,6,7,6,5,5,5,5,5,4,4,4,4,4,5,6,6,5,7,8,8,9,9,9,8,8,7,6,6,4,3,3,3,4,5,8,9,11,12,...,13,13,14,15,16,16,19,20,23,24,25,27,27,27,27,28,32,38,42,47,51,57,61,62,66,70,75,77,76,72,72,69,66,67,66,69,68,65,61,64
1,62,60,55,55,54,52,51,50,49,45,44,45,45,43,45,46,45,47,50,50,51,51,53,51,47,43,39,31,23,17,15,13,12,13,13,15,20,22,23,24,...,25,26,27,28,27,25,28,26,27,25,24,22,22,21,21,22,26,30,31,34,34,37,40,40,40,40,38,36,35,34,34,33,31,31,31,31,31,31,27,28
2,29,29,29,30,28,27,28,26,22,20,19,18,19,19,17,16,15,14,16,17,17,16,16,15,14,15,12,8,6,4,2,1,1,1,1,1,2,3,3,3,...,3,2,3,3,4,5,5,6,7,9,11,11,13,13,14,16,22,23,23,28,29,32,34,37,38,36,37,35,34,34,30,27,28,28,25,23,21,19,16,15
3,13,12,12,11,12,12,12,10,10,10,10,10,9,11,11,11,9,9,11,11,10,11,11,11,10,9,8,6,5,4,3,3,4,4,5,5,6,8,7,7,...,13,13,13,14,14,11,11,9,8,9,9,8,9,11,9,9,9,11,14,19,21,23,22,22,22,25,27,26,24,22,21,22,22,22,23,23,24,25,24,23
4,21,20,19,16,15,14,13,11,11,11,10,10,11,11,11,11,10,11,13,14,15,17,18,18,19,20,19,16,13,10,9,6,5,4,4,4,4,4,4,5,...,5,5,8,9,9,14,15,19,21,24,29,22,21,20,18,18,18,15,16,14,13,23,30,29,28,31,36,32,31,31,34,33,31,31,31,33,33,34,33,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,116,117,117,113,114,114,111,114,114,113,111,110,111,110,105,105,104,104,104,109,108,106,106,106,101,95,87,87,83,72,71,71,69,66,68,68,72,74,75,74,...,92,96,97,92,92,90,89,90,84,82,76,74,73,77,85,91,98,106,111,117,119,121,124,127,124,126,126,126,124,117,114,116,112,108,104,99,96,96,92,92
393,91,92,91,92,92,90,90,90,93,93,97,99,100,101,102,100,104,106,105,106,108,108,105,103,101,95,88,78,69,64,56,51,46,43,43,43,40,36,33,31,...,24,25,25,27,26,26,27,27,27,27,31,35,41,48,52,56,61,65,68,70,74,75,78,79,78,76,75,71,67,64,61,58,58,58,58,58,57,58,58,60
394,60,59,58,55,52,50,48,45,43,41,39,36,34,34,32,33,34,34,34,33,32,35,37,37,39,39,37,35,31,27,22,18,14,13,14,12,13,15,15,15,...,28,29,29,31,32,35,37,36,36,35,34,37,41,45,49,56,62,69,71,72,71,70,68,67,63,61,58,58,56,54,53,50,47,46,45,45,44,46,46,45
395,44,44,40,39,37,36,34,33,31,28,26,24,24,24,23,21,20,19,19,18,18,18,18,16,16,16,14,14,13,10,8,6,4,4,3,3,3,4,4,4,...,4,4,5,5,5,5,5,5,5,6,6,6,7,8,9,10,11,12,12,12,13,12,12,12,13,13,13,13,13,13,13,12,12,12,12,11,11,12,12,11


In [19]:
dataset = dataframe.values
dataset.shape

(397, 96)

In [20]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [21]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

265 132


In [22]:
import numpy as np
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [23]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [29]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainX.shape
testX.shape

(130, 1, 1)

In [25]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
263/263 - 2s - loss: 0.0636
Epoch 2/100
263/263 - 0s - loss: 0.0430
Epoch 3/100
263/263 - 0s - loss: 0.0414
Epoch 4/100
263/263 - 0s - loss: 0.0405
Epoch 5/100
263/263 - 0s - loss: 0.0398
Epoch 6/100
263/263 - 0s - loss: 0.0393
Epoch 7/100
263/263 - 0s - loss: 0.0390
Epoch 8/100
263/263 - 0s - loss: 0.0390
Epoch 9/100
263/263 - 0s - loss: 0.0388
Epoch 10/100
263/263 - 0s - loss: 0.0387
Epoch 11/100
263/263 - 0s - loss: 0.0387
Epoch 12/100
263/263 - 0s - loss: 0.0386
Epoch 13/100
263/263 - 0s - loss: 0.0387
Epoch 14/100
263/263 - 0s - loss: 0.0386
Epoch 15/100
263/263 - 0s - loss: 0.0386
Epoch 16/100
263/263 - 0s - loss: 0.0386
Epoch 17/100
263/263 - 0s - loss: 0.0387
Epoch 18/100
263/263 - 0s - loss: 0.0386
Epoch 19/100
263/263 - 0s - loss: 0.0386
Epoch 20/100
263/263 - 0s - loss: 0.0386
Epoch 21/100
263/263 - 0s - loss: 0.0385
Epoch 22/100
263/263 - 0s - loss: 0.0384
Epoch 23/100
263/263 - 0s - loss: 0.0384
Epoch 24/100
263/263 - 0s - loss: 0.0384
Epoch 25/100
263/263 - 0s

In [33]:
# make predictions
trainPredict = model.predict(trainX)
trainPredict.shape
testPredict = model.predict(testX)
testPredict.shape
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# calculate mean absolute error
trainScore = mean_absolute_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f MAE' % (trainScore))
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('Test Score: %.2f MAE' % (testScore))

ValueError: ignored

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.xlabel('Days')
plt.ylabel('energy generated')
plt.show()